In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tifffile as tf
import pickle
from datetime import datetime
import os
from skimage.color import label2rgb
import sys
sys.path.append("../../")
from functions_EDX import *

### EDX colors of interest

In [2]:
indices = [3,4,6,1,9,7]
colors = [[1,0,0], [0,1,0], [0,0,1], [0,1,1], [1,0,1], [1,1,0]]

### load the stack of haadfs, SAM masks, abundance maps

In [3]:
# haadf stack
haadf_stack = tf.imread("../../../../primary_data/main_mosaic_5by6_haadf.tiff")  

# location of SAM masks
masks_path = '../../../../primary_data/main_mosaic_6by5/SAM_masks/'

# abundance maps
abundance_maps = np.load("../../../../primary_data/abundance_maps.npz")['arr_0']
print('abundance maps shape',abundance_maps.shape)

abundance maps shape (11, 1024, 1024, 30)


### Output path for the tiffs and options

In [4]:
area_thresh = 500000
overlay_haadf = True
alpha = 0.35
out_dir = '../../../../primary_data/main_mosaic_6by5/SAM_EDX_masks_tiff/'

In [ ]:
sub_dir = 'haadf_%s_alpha_%02d_maxarea_%07d' % (str(overlay_haadf),int(alpha*100),area_thresh)
try:
    os.mkdir(os.path.join(out_dir,sub_dir))
    outpath = os.path.join(out_dir,sub_dir)
except:
    print('These options already exist.')
    sys.exit(1)
        
for tile_idx in range(abundance_maps.shape[3]):
    haadf = haadf_stack[tile_idx,:,:] 
    abundance_tile = abundance_maps[indices,:,:,tile_idx]
    
    # masks
    # Get the masks
    file = open(os.path.join(masks_path,'tile_%02d.pkl' % tile_idx),'rb')
    masks = pickle.load(file)
    
    if overlay_haadf:
        SamEDXImg,img_clr_idx = show_anns_EDX(masks,abundance_tile,colors=colors,display=False,alpha = alpha,area_thresh=area_thresh)
        #SamEDXImg = cv2.addWeighted(cv2.cvtColor(haadf,cv2.COLOR_GRAY2RGBA), 1-alpha,(SamEDXImg[:,:,:]*255).astype('uint8'), alpha, 0)
        Img = label2rgb(img_clr_idx, image=haadf,colors=colors,kind='overlay',alpha = alpha,bg_label=-1)
        tf.imwrite(os.path.join(outpath,'tile_%02d.tiff' % tile_idx),Img)
    else:
        SamEDXImg = show_anns_EDX(masks,abundance_tile,colors=colors,display=False,alpha = alpha,area_thresh=area_thresh)
        tf.imwrite(os.path.join(outpath,'tile_%02d.tiff' % tile_idx),(SamEDXImg[:,:,:]*255).astype('uint8'))
    
    
    print(tile_idx)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
